In [96]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
r.status_code

200

In [33]:
soup = BeautifulSoup(r.text,'lxml')

In [41]:
tbodys = soup.find('tbody')

In [59]:
#table header capture
header=tbodys.find_all('th')
header=[x.text.strip() for x in header]

In [46]:
rows = tbodys.find_all('tr')
table=[]
for row in rows[1:]:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    table.append(cols)


In [97]:
df=pd.DataFrame(table,columns=header)

* drop missing borough

In [98]:
df=df[~(df['Borough']=='Not assigned')] 

In [99]:
postcode=df.Postcode
borough=df.Borough
Neighbourhood=df.Neighbourhood

* Missing neighbourhood replaced by borough

In [100]:
for i in range(len(borough)):
    if Neighbourhood.iloc[i]=='Not assigned':
        Neighbourhood.iloc[i]=borough.iloc[i]

In [101]:
postcode=df.Postcode.tolist()
borough=df.Borough.tolist()
Neighbourhood=df.Neighbourhood.tolist()

* Neighbourhood list groupby postcode

In [102]:
new_table=[]
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcode) :   
    if postcode_i not in postcode_explored :
        nbds = Neighbourhood[index_i]
        for index_f, postcode_f in enumerate(postcode) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + Neighbourhood[index_f] # Concatenating the neighbourhood names
        new_table.append([postcode_i, borough[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

In [103]:
df=pd.DataFrame(new_table,columns=header)

In [104]:
df.shape

(103, 3)